In [1]:
import pandas as pd
import numpy as np
import logging
import sys
from datetime import datetime

# a little hacky, but works if you don't want to actually install the
# custom packages 
sys.path.append('../')
from uniswapv3_simulator.pool import Uniswapv3Pool
from uniswapv3_simulator.utils import pool_init_price, solve_for_liquidity_delta
from uniswapv3_simulator.math import tick_to_sqrt_price, sqrt_price_to_tick
from utils import amount_to_float

In [2]:
POOL = 'WETH-USDT-500'
MAX_DATE = '2022-01-28'

timestamp = datetime.now().strftime('%y%m%d%H%M%S')
logging.basicConfig(level=logging.INFO,
                    filename=f'./logs/{POOL}_{timestamp}.log')
logging.getLogger('uniswap-v3').setLevel(logging.DEBUG)
logging.getLogger('covalent_api').setLevel(logging.DEBUG)

In [3]:
data = pd.read_pickle('../data/pool_data_clean.pickle')
swaps = data[POOL]['swaps']
liquidity = data[POOL]['liquidity']

In [4]:
swaps.info()
swaps.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735956 entries, 0 to 735955
Data columns (total 18 columns):
 #   Column                          Non-Null Count   Dtype              
---  ------                          --------------   -----              
 0   swap_time                       735956 non-null  datetime64[ns, UTC]
 1   tx_hash                         735956 non-null  string             
 2   token_0_amount                  735956 non-null  float64            
 3   token_1_amount                  735956 non-null  float64            
 4   pool_address                    735956 non-null  string             
 5   price_tick                      735956 non-null  int64              
 6   price_tick_adjusted             735956 non-null  float64            
 7   contract_address_token_0        735956 non-null  string             
 8   contract_name_token_0           735956 non-null  string             
 9   contract_ticker_symbol_token_0  735956 non-null  string             
 

,swap_time,tx_hash,token_0_amount,token_1_amount,pool_address,price_tick,price_tick_adjusted,contract_address_token_0,contract_name_token_0,contract_ticker_symbol_token_0,contract_decimals_token_0,contract_address_token_1,contract_name_token_1,contract_ticker_symbol_token_1,contract_decimals_token_1,pool_fee,pool_tick_spacing,pool_deploy_time
0,2022-01-29 23:41:35+00:00,0xf5d97847028540428cca1defd028ffb9026a03f7517f...,1.482148,-3836.514062,0x11b815efb8f581194ae79006d24e0d814b7697f6,-197728,2589.4731,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,500,10,2021-05-05 21:46:12+00:00
1,2022-01-29 23:41:18+00:00,0x39fc556ad45d45cf69a4636812e75566548bee99f6ab...,-7.254661,18795.921420,0x11b815efb8f581194ae79006d24e0d814b7697f6,-197727,2589.7320,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,500,10,2021-05-05 21:46:12+00:00
2,2022-01-29 23:40:21+00:00,0x638a0c8ead471a2119fd1c16342789b613e61703d7bd...,1.403955,-3633.554617,0x11b815efb8f581194ae79006d24e0d814b7697f6,-197729,2589.2142,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,500,10,2021-05-05 21:46:12+00:00
3,2022-01-29 23:39:15+00:00,0x20fb59d67d772cf31078b080ac8bc94e208811e47fe4...,20.000000,-51776.324338,0x11b815efb8f581194ae79006d24e0d814b7697f6,-197729,2589.2142,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,500,10,2021-05-05 21:46:12+00:00
4,2022-01-29 23:37:57+00:00,0x11da659ede544b89cc545798be0104ce64b375eefe11...,4.309660,-11160.497318,0x11b815efb8f581194ae79006d24e0d814b7697f6,-197723,2590.7681,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,500,10,2021-05-05 21:46:12+00:00


In [5]:
liquidity.info()
liquidity.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6378 entries, 0 to 6377
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype              
---  ------                          --------------  -----              
 0   txn_time                        6378 non-null   datetime64[ns, UTC]
 1   tx_hash                         6378 non-null   string             
 2   token_0_amount                  6378 non-null   float64            
 3   token_1_amount                  6378 non-null   float64            
 4   pool_address                    6378 non-null   string             
 5   liquidity_event                 6378 non-null   string             
 6   price_tick_lower                6378 non-null   int64              
 7   price_tick_upper                6378 non-null   int64              
 8   price_tick_lower_adjusted       6378 non-null   float64            
 9   price_tick_upper_adjusted       6378 non-null   float64            
 10  contract_add

,txn_time,tx_hash,token_0_amount,token_1_amount,pool_address,liquidity_event,price_tick_lower,price_tick_upper,price_tick_lower_adjusted,price_tick_upper_adjusted,...,contract_decimals_token_0,contract_address_token_1,contract_name_token_1,contract_ticker_symbol_token_1,contract_decimals_token_1,event_name,pool_fee,pool_tick_spacing,pool_deploy_time,liquidity
0,2022-01-29 13:45:44+00:00,0x94e2221c60c64b80dfad82980d1812699d5882080adf...,0.000000,87467.069461,0x11b815efb8f581194ae79006d24e0d814b7697f6,REMOVE_LIQUIDITY,-199220,-197730,2230.5805,2588.9553,...,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,Burn,500,10,2021-05-05 21:46:12+00:00,-0.023946
1,2022-01-29 11:45:42+00:00,0x15712ba6f8a2ee3034ba04f6da308f36a55e3de55a5d...,3.544193,78359.898071,0x11b815efb8f581194ae79006d24e0d814b7697f6,ADD_LIQUIDITY,-199220,-197730,2230.5805,2588.9553,...,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,Mint,500,10,2021-05-05 21:46:12+00:00,0.023946
2,2022-01-29 09:32:56+00:00,0xcfed9839480120c67418e4c8572ff8b948d85df3603b...,0.000000,2514.450974,0x11b815efb8f581194ae79006d24e0d814b7697f6,REMOVE_LIQUIDITY,-199530,-198560,2162.4967,2382.7578,...,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,Burn,500,10,2021-05-05 21:46:12+00:00,-0.001088
3,2022-01-29 09:32:56+00:00,0xcfed9839480120c67418e4c8572ff8b948d85df3603b...,0.041104,497.281442,0x11b815efb8f581194ae79006d24e0d814b7697f6,REMOVE_LIQUIDITY,-199530,-197590,2162.4967,2625.4537,...,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,Burn,500,10,2021-05-05 21:46:12+00:00,-0.000127
4,2022-01-29 08:58:33+00:00,0xd85cc8c477c474bc714b662f3def1cec0a7a40a15532...,1.100000,7594.658343,0x11b815efb8f581194ae79006d24e0d814b7697f6,ADD_LIQUIDITY,-199360,-197310,2199.5715,2700.0015,...,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,Mint,500,10,2021-05-05 21:46:12+00:00,0.002071


In [6]:
adds = liquidity.loc[liquidity['liquidity_event'] == 'ADD_LIQUIDITY', :].copy()
adds = adds.sort_values('txn_time').reset_index(drop=True)
adds.head()

,txn_time,tx_hash,token_0_amount,token_1_amount,pool_address,liquidity_event,price_tick_lower,price_tick_upper,price_tick_lower_adjusted,price_tick_upper_adjusted,...,contract_decimals_token_0,contract_address_token_1,contract_name_token_1,contract_ticker_symbol_token_1,contract_decimals_token_1,event_name,pool_fee,pool_tick_spacing,pool_deploy_time,liquidity
0,2021-05-05 21:46:12+00:00,0x5399bd3a8fa539a1899af6b3c10a526d07e5c371a0c0...,0.500000,1496.807588,0x11b815efb8f581194ae79006d24e0d814b7697f6,ADD_LIQUIDITY,-198080,-191150,2499.9136,4998.9182,...,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,Mint,500,10,2021-05-05 21:46:12+00:00,0.000172
1,2021-05-05 22:18:51+00:00,0x68467f65460cc8a71eb042ad2f9c05e6f57e0927bdf1...,3.623997,11528.727666,0x11b815efb8f581194ae79006d24e0d814b7697f6,ADD_LIQUIDITY,-196260,-193380,2998.9045,3999.7533,...,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,IncreaseLiquidity,500,10,2021-05-05 21:46:12+00:00,0.002945
2,2021-05-05 22:35:50+00:00,0xd2aea79560102a9095113d13ae21ea596051249c5a62...,0.021262,100.000000,0x11b815efb8f581194ae79006d24e0d814b7697f6,ADD_LIQUIDITY,-200310,-191150,2000.2403,4998.9182,...,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,Mint,500,10,2021-05-05 21:46:12+00:00,0.000007
3,2021-05-05 22:55:12+00:00,0x7262684c16d0a792deeec28c732926c49d5dde345c74...,0.026318,100.000000,0x11b815efb8f581194ae79006d24e0d814b7697f6,ADD_LIQUIDITY,-196260,-193380,2998.9045,3999.7533,...,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,IncreaseLiquidity,500,10,2021-05-05 21:46:12+00:00,0.000023
4,2021-05-05 23:03:39+00:00,0xb0b5e71882849618a04225a024f49e8520ae6a3b5b01...,0.000095,1.000000,0x11b815efb8f581194ae79006d24e0d814b7697f6,ADD_LIQUIDITY,-194800,-194620,3470.2956,3533.3233,...,18,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,IncreaseLiquidity,500,10,2021-05-05 21:46:12+00:00,0.000003


In [7]:
first_add_hash = adds.at[0, 'tx_hash']
print(f'First liquidity add hash: {first_add_hash}')

First liquidity add hash: 0x5399bd3a8fa539a1899af6b3c10a526d07e5c371a0c0bbcfde7c6cbffe88a59f


In [8]:
# from https://etherscan.io/tx/0x5399bd3a8fa539a1899af6b3c10a526d07e5c371a0c0bbcfde7c6cbffe88a59f#eventlog
liquidity_delta = amount_to_float('172485837547939', 18)  # belive all liquidity amounts use 18 decimals
assert liquidity_delta == adds.at[0, 'liquidity']

token0 = adds.at[0, 'token_0_amount']
token1 = adds.at[0, 'token_1_amount']
tick_lower = adds.at[0, 'price_tick_lower']
tick_upper = adds.at[0, 'price_tick_upper']
token0_decimals = adds.at[0, 'contract_decimals_token_0']
token1_decimals = adds.at[0, 'contract_decimals_token_1']

init_price = pool_init_price(token0, token1, tick_upper, tick_lower, liquidity_delta,
                             token0_decimals, token1_decimals)

token0_symb = liquidity.at[0, 'contract_ticker_symbol_token_0']
token1_symb = liquidity.at[0, 'contract_ticker_symbol_token_1']
print(f'Pool initial price ({token1_symb}/{token0_symb}): {init_price:,.12e}')
print(f'Pool initial price ({token0_symb}/{token1_symb}): {1 / init_price:,.2e}')

Pool initial price (USDT/WETH): 3.442989611235e-09
Pool initial price (WETH/USDT): 2.90e+08


In [9]:
sqrt_price_x96 = 4648870407266953854345730
etherscan_price = sqrt_price_x96 ** 2 / 2 ** 192
print(f"Calculated initial price: {init_price:.12e}")
print(f"Price per Etherscan:      {etherscan_price:.12e}")

Calculated initial price: 3.442989611235e-09
Price per Etherscan:      3.442989611229e-09


In [10]:
cols = ['tx_hash', 'txn_time', 'liquidity_event']
liqu_txn = liquidity.loc[:, cols].copy()
liqu_txn.reset_index(drop=False, inplace=True)
liqu_txn.rename(columns={'liquidity_event': 'event', 'index': 'orig_idx'}, inplace=True)

cols = ['tx_hash', 'swap_time']
swap_txn = swaps.loc[:, cols].copy()
swap_txn.reset_index(drop=False, inplace=True)
swap_txn.rename(columns={'swap_time': 'txn_time', 'index': 'orig_idx'}, inplace=True)
swap_txn['event'] = 'SWAP'

all_txn = pd.concat([liqu_txn, swap_txn], axis=0)
all_txn = all_txn.sort_values('txn_time').reset_index(drop=True)
all_txn.drop(all_txn.index[all_txn['txn_time'] > MAX_DATE], axis=0, inplace=True)

all_txn

,orig_idx,tx_hash,txn_time,event
0,6377,0x5399bd3a8fa539a1899af6b3c10a526d07e5c371a0c0...,2021-05-05 21:46:12+00:00,ADD_LIQUIDITY
1,6376,0x68467f65460cc8a71eb042ad2f9c05e6f57e0927bdf1...,2021-05-05 22:18:51+00:00,ADD_LIQUIDITY
2,735955,0xa82fa5e82d2cb004852b028ae2fa6983c3a09b752ec8...,2021-05-05 22:23:23+00:00,SWAP
3,6375,0xdadf7156061bf60873adc3e363515490eb66af239eef...,2021-05-05 22:30:21+00:00,REMOVE_LIQUIDITY
4,6374,0xdadf7156061bf60873adc3e363515490eb66af239eef...,2021-05-05 22:30:21+00:00,REMOVE_LIQUIDITY
...,...,...,...,...
738966,3336,0x76c83388f2f10413e177423928b3d76780640cf00df1...,2022-01-27 23:53:43+00:00,SWAP
738967,3335,0x041e81cfc04952eb41bb6bc6953948a0fdd22eb0c4ae...,2022-01-27 23:53:59+00:00,SWAP
738968,3334,0x7f9ebfeafd5bfdd281da0576f180148af257ca613a2b...,2022-01-27 23:54:06+00:00,SWAP
738969,3333,0x97754d201fd42e4b977a0a691fc5a242041c326abdd3...,2022-01-27 23:58:44+00:00,SWAP


In [11]:
CHECKS_ON = True
# need to think about appropriate error tolerances
# TODO: maybe base these tolerances on the average transaction size?
TOKEN0_TOLS = {'atol': 1e-6, 'rtol': 1e-4}
TOKEN1_TOLS = {'atol': 1e-6, 'rtol': 1e-4}
LIQUIDITY_TOLS = {'atol': 1e-6, 'rtol': 1e-4}

In [12]:
all_txn = all_txn.iloc[:50].copy()

In [13]:
fee = liquidity.at[0, 'pool_fee'] / 1e+6
tick_spacing = liquidity.at[0, 'pool_tick_spacing']
pool = Uniswapv3Pool(fee, tick_spacing, init_price,
                     token0_decimals=token0_decimals,
                     token1_decimals=token1_decimals)
print(f'{pool}')

position_id = 'LP123'
results = []
for i, row in all_txn.iterrows():
    logging.info(f'Transaction {i}.')
    txn = row['event']
    idx = row['orig_idx']
    
    if 'LIQUIDITY' in txn:
        token0 = liquidity.at[idx, 'token_0_amount']
        token1 = liquidity.at[idx, 'token_1_amount']
        if txn == 'REMOVE_LIQUIDITY':
            token0 = -1 * token0
            token1 = -1 * token1
        
        tick_lower = liquidity.at[idx, 'price_tick_lower']
        tick_upper = liquidity.at[idx, 'price_tick_upper']
        liquidity_delta = liquidity.at[idx, 'liquidity']
        
        if pd.isnull(liquidity_delta):
            liquidity_delta = solve_for_liquidity_delta(
                token0, 
                token1, 
                tick_lower, 
                tick_upper, 
                pool.sqrt_price,
                token0_decimals,
                token1_decimals
            )
        elif CHECKS_ON:
            ld_calc = solve_for_liquidity_delta(
                token0, 
                token1, 
                tick_lower, 
                tick_upper, 
                pool.sqrt_price,
                token0_decimals,
                token1_decimals
            )
            assert np.isclose(liquidity_delta, ld_calc, **LIQUIDITY_TOLS), (
                f'Calculated liquidity_delta {ld_calc:,.12e} does '
                f'not match liquidity_delta per the data {liquidity_delta:,.12e}.'
            )
            
        position = pool.position_map[(position_id, tick_lower, tick_upper)]
        if liquidity_delta < 0:
            # If the liquidity_delta is very, very close to the position's total
            # liquidity, set liquidity_delta to the total liquidity to completely
            # close out the position
            if np.isclose(-position.liquidity, liquidity_delta):
                liquidity_delta = -position.liquidity
            # we also make sure that liquidity_delta cannot be less than the 
            # position's total liquidity
            if liquidity_delta < 0:
                liquidity_delta = max(liquidity_delta, -position.liquidity)
                
        token0_calc, token1_calc = pool.set_position(
            position_id, 
            tick_lower,
            tick_upper,
            liquidity_delta
        )
    elif txn == 'SWAP':
        token0 = swaps.at[idx, 'token_0_amount']
        token1 = swaps.at[idx, 'token_1_amount']
        
        token = 0 if token0 > 0 else 1
        tokens_in = token0 if token == 0 else token1
        
        token0_calc, token1_calc = pool.swap(token, tokens_in)
    else:
        raise ValueError(f'{txn} is not a valid transaction type.')
    
    if CHECKS_ON:
        assert np.isclose(token0, -token0_calc, **TOKEN0_TOLS), (
            f'Transaction {i:,}: token0 output {-token0_calc:,.12e} does not '
            f'match token0 in the data {token0:,.12e}.'
        )
        assert np.isclose(token1, -token1_calc, **TOKEN1_TOLS), (
            f'Transaction {i:,}: token1 output {-token1_calc:,.12e} does not '
            f'match token1 in the data {token1:,.12e}.'
        )
    
    results.append({
        'sqrt_price': pool.sqrt_price, 
        'liquidity': pool.liquidity
    })
    print(f'Completed transaction {i}.')

Pool(price=3.442990e+03, liquidity=0.000000e+00, fee=0.05%)
Completed transaction 0.
Completed transaction 1.
Completed transaction 2.
Completed transaction 3.
Completed transaction 4.
Completed transaction 5.
Completed transaction 6.
Completed transaction 7.
Completed transaction 8.
Completed transaction 9.
Completed transaction 10.
Completed transaction 11.
Completed transaction 12.
Completed transaction 13.
Completed transaction 14.
Completed transaction 15.
Completed transaction 16.
Completed transaction 17.
Completed transaction 18.
Completed transaction 19.
Completed transaction 20.
Completed transaction 21.
Completed transaction 22.
Completed transaction 23.
Completed transaction 24.
Completed transaction 25.
Completed transaction 26.
Completed transaction 27.
Completed transaction 28.
Completed transaction 29.
Completed transaction 30.
Completed transaction 31.
Completed transaction 32.
Completed transaction 33.
Completed transaction 34.
Completed transaction 35.
Completed tran